In [1]:

import tensorflow as tf
import pickle
import os 
import cv2
import numpy as np
print(tf.__version__)



2.3.3


In [2]:
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [3]:
! export TF_FORCE_GPU_ALLOW_GROWTH=true

In [4]:
IMAGE_SIZE = (224,224,3)

In [5]:
physical_devices = tf.config.list_physical_devices('GPU')
for device in physical_devices:
    tf.config.experimental.set_memory_growth(device, True)

In [6]:
###
### Use all GPUs.
###
strategy = tf.distribute.MirroredStrategy()
print()
print()
print()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')



Number of devices: 8


In [7]:
def make_model_2():
    def build_convnet(shape=None):
        momentum = 0.9
        model = tf.keras.Sequential()
        model.add(tf.keras.layers.Conv2D(64, (3,3), input_shape=shape[1:], padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        model.add(tf.keras.layers.MaxPool2D())

        model.add(tf.keras.layers.Conv2D(128, (3,3), padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        model.add(tf.keras.layers.MaxPool2D())

        model.add(tf.keras.layers.Conv2D(256, (3,3), padding='same', activation='linear'))
        model.add(tf.keras.layers.BatchNormalization(momentum=momentum))
        model.add(tf.keras.layers.Activation('relu'))

        # flatten
        model.add(tf.keras.layers.GlobalMaxPool2D())
        return model
    shape = (90, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_SIZE[2])
    print('Train data shape: ', shape)

    convnet = build_convnet(shape)

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.TimeDistributed(convnet, input_shape=shape))
    model.add(tf.keras.layers.LSTM(64))

    model.add(tf.keras.layers.Dense(256, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(128, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(64, activation='relu'))
    model.add(tf.keras.layers.Dropout(.5))
    model.add(tf.keras.layers.Dense(16, activation='relu'))

    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))
    
    return model
        

In [24]:
def make_model(): 
    inp =  tf.keras.layers.Input((None, IMAGE_SIZE[0], IMAGE_SIZE[1], IMAGE_SIZE[2])) # , ragged=True
    
    mobilenet_model = tf.keras.applications.mobilenet_v2.MobileNetV2(
        include_top=False, weights='imagenet', pooling='max', classes=2, input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)
    )
    
    for k,v in mobilenet_model._get_trainable_state().items():
        k.trainable = False
    
#     mobilenet_model = tf.keras.models.Sequential()
#     mobilenet_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3)))
#     mobilenet_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
#     mobilenet_model.add(tf.keras.layers.Conv2D(16, (3, 3), activation='relu'))
# #     mobilenet_model.add(tf.keras.layers.MaxPooling2D((2, 2)))
# #     mobilenet_model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
#     mobilenet_model.add(tf.keras.layers.Flatten())
    
    x = tf.keras.layers.TimeDistributed(mobilenet_model)(inp)
    x = tf.keras.layers.LSTM(64, return_sequences=False)(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    out = tf.keras.layers.Dense(1, activation = 'sigmoid')(x)

    model = tf.keras.Model(inp, out)

    model.compile(loss = tf.keras.losses.BinaryCrossentropy(), optimizer = tf.keras.optimizers.Adam(learning_rate=0.01), metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

    return model

In [25]:
# model = make_model()

# model.build(input_shape=IMAGE_SIZE)
# model.summary()

Model: "functional_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_7 (InputLayer)         [(None, None, 224, 224, 3 0         
_________________________________________________________________
time_distributed_3 (TimeDist (None, None, 1280)        2257984   
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                344320    
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 65        
Total params: 2,602,369
Trainable params: 344,385
Non-trainable params: 2,257,984
_________________________________________________________________


In [29]:
all_accs = []
all_pres = []
all_recs = []
all_f1s = []

In [ ]:
with strategy.scope():

    with open("file_names_folds.pkl", 'rb') as f: 
        SEEDS, FOLD_FILES = pickle.load(f)

        for fold in FOLD_FILES:
            train_files = [a.strip('_') for a in fold['train']]
            test_files = [a.strip('_') for a in fold['test']]

            X_train = []
            X_test = []

            y_train = []
            y_test = []

            for filename in train_files:
                filename_int = int(filename.split('.mp4')[0])

                if filename_int <= 115:
                    curr_y = 1
                    subdir_name = 'armflapping'
                else:
                    curr_y = 0
                    subdir_name = 'control'

                curr_x = []
                for frame in os.listdir('behavior_data/' + subdir_name + '/' + filename):

                    frame_num = int(frame.split('.')[0])
                    if frame_num > 90:
                        continue

                    image = cv2.imread('behavior_data/' + subdir_name + '/' + filename + '/' + frame)
                    try:
                        image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
                    except:
                        continue

                    image = cv2.resize(image, (224, 224))
                    curr_x.append(image)

                len_data = len(os.listdir('behavior_data/' + subdir_name + '/' + filename))
                if len_data < 90:
                    for abc in range(len_data, 90):
                        curr_x.append(np.zeros((224, 224, 3)))

                curr_x = np.array(curr_x)

                X_train.append(curr_x)
                y_train.append(curr_y)

            for filename in test_files:
                filename_int = int(filename.split('.mp4')[0])

                if filename_int <= 115:
                    curr_y = 1
                    subdir_name = 'armflapping'
                else:
                    curr_y = 0
                    subdir_name = 'control'

                curr_x = []
                for frame in os.listdir('behavior_data/' + subdir_name + '/' + filename):

                    frame_num = int(frame.split('.')[0])
                    if frame_num > 90:
                        continue

                    image = cv2.imread('behavior_data/' + subdir_name + '/' + filename + '/' + frame)
                    try:
                        image = image.reshape((image.shape[0], image.shape[1], image.shape[2]))
                    except:
                        continue

                    image = cv2.resize(image, (224, 224))
                    curr_x.append(image)

                len_data = len(os.listdir('behavior_data/' + subdir_name + '/' + filename))
                if len_data < 90:
                    for abc in range(len_data, 90):
                        curr_x.append(np.zeros((224, 224, 3)))

                curr_x = np.array(curr_x)

                X_test.append(curr_x)
                y_test.append(curr_y)

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

            model = make_model()

            model.compile(loss = tf.keras.losses.BinaryCrossentropy(), 
                          optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001), 
                          metrics = ['accuracy', tf.keras.metrics.Precision(), tf.keras.metrics.Recall()])

            history = model.fit(X_train, 
                                y_train, 
                                #validation_data = (X_test, y_test), 
                                batch_size = 16,
                                epochs = 60)


            ###
            ### EVALUATE HERE!!!!!
            ###
            
            predictions = []
            trues = []
            for i in range(X_test.shape[0]):
                X_to_predict = np.array([X_test[i]])
                pred = model(X_to_predict)
                print(pred)
                true = y_test[i]
                if pred < 0.5:
                    predictions.append(0)
                else:
                    predictions.append(1)
                trues.append(true)

            acc = accuracy_score(trues, predictions)
            pre = precision_score(trues, predictions)
            rec = recall_score(trues, predictions)
            f1 = f1_score(trues, predictions)
            
            print('\n\n\n\n\n\n\n  ', acc, pre, rec, f1, ' \n\n\n\n\n\n\n')
            
            all_accs.append(acc)
            all_pres.append(pre)
            all_recs.append(rec)
            all_f1s.append(f1)


(80, 90, 224, 224, 3) (20, 90, 224, 224, 3) (80,) (20,)
Epoch 1/60
INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


5/5 [==============================] - 1s 106ms/step - loss: 0.6823 - accuracy: 0.5625 - precision_9: 0.5926 - recall_9: 0.4000
Epoch 2/60
5/5 [==============================] - 0s 99ms/step - loss: 0.6259 - accuracy: 0.6500 - precision_9: 0.6579 - recall_9: 0.6250
Epoch 3/60
5/5 [==============================] - 1s 100ms/step - loss: 0.5400 - accuracy: 0.7875 - precision_9: 0.8485 - recall_9: 0.7000
Epoch 4/60
5/5 [==============================] - 1s 101ms/step - loss: 0.5374 - accuracy: 0.7750 - precision_9: 0.7750 - recall_9: 0.7750
Epoch 5/60
5/5 [==============================] - 1s 106ms/step - loss: 0.4892 - accuracy: 0.8000 - precision_9: 0.8333 - recall_9: 0.7500
Epoch 6/60
5/5 [==============================] - 1s 100ms/step - loss: 0.4282 - accuracy: 0.8500 - precision_9: 0.9118 - recall_9: 0.7750
Epoch 7/60
5/5 [==============================] - 1s 102ms/step - loss: 0.4070 - accuracy: 0.8250 - precision_9: 0.8824 - recall_9: 0.7500
Epoch 8/60
5/5 [=======================

INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


INFO:tensorflow:batch_all_reduce: 5 all-reduces with algorithm = nccl, num_packs = 1


5/5 [==============================] - 1s 113ms/step - loss: 0.7480 - accuracy: 0.4750 - precision_11: 0.4839 - recall_11: 0.7500
Epoch 2/60
5/5 [==============================] - 1s 102ms/step - loss: 0.5448 - accuracy: 0.7750 - precision_11: 0.7292 - recall_11: 0.8750
Epoch 3/60
5/5 [==============================] - 0s 99ms/step - loss: 0.4894 - accuracy: 0.7750 - precision_11: 0.7500 - recall_11: 0.8250
Epoch 4/60
5/5 [==============================] - 0s 99ms/step - loss: 0.4375 - accuracy: 0.8125 - precision_11: 0.7907 - recall_11: 0.8500
Epoch 5/60
5/5 [==============================] - 1s 102ms/step - loss: 0.4692 - accuracy: 0.8125 - precision_11: 0.7778 - recall_11: 0.8750
Epoch 6/60
5/5 [==============================] - 0s 98ms/step - loss: 0.3979 - accuracy: 0.9125 - precision_11: 0.9459 - recall_11: 0.8750
Epoch 7/60
5/5 [==============================] - 0s 98ms/step - loss: 0.3741 - accuracy: 0.9125 - precision_11: 0.9714 - recall_11: 0.8500
Epoch 8/60
5/5 [============

In [ ]:
from scipy import stats
print('Acc: ', stats.describe(all_accs))
print('Pre: ', stats.describe(all_pres))
print('Rec: ', stats.describe(all_recs))
print('F1: ', stats.describe(all_f1s))

In [27]:
'''
THIS IS HOW WE WILL HAVE TO EVALUATE PREDICTIONS!!!!!

https://stackoverflow.com/questions/56711354/tensorflow-v2-cancellederror-opstatefulpartitionedcall
'''

from sklearn.metrics import recall_score, precision_score, accuracy_score, f1_score

with strategy.scope():
    predictions = []
    trues = []
    for i in range(X_test.shape[0]):
        X_to_predict = np.array([X_test[i]])
        pred = model(X_to_predict)
        print(pred)
        true = y_test[i]
        if pred < 0.5:
            predictions.append(0)
        else:
            predictions.append(1)
        trues.append(true)
        
    print('Accuracy: ', accuracy_score(trues, predictions))
    print('Precision: ', precision_score(trues, predictions))
    print('Recall: ', recall_score(trues, predictions))
    print('F1 Score: ', f1_score(trues, predictions))
    

tf.Tensor([[0.8247404]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9714775]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9940625]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.99450547]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9829216]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9581735]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9916317]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9885062]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.81362104]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.9727558]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.05049171]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.4205745]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.1080795]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.04499972]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.0348262]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.04339964]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.03726655]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.01246449]], shape=(1, 1), dtype=float32)
tf.Tensor([[0.6749961